In [4]:
import vnstock as vn
import ta
import pandas as pd
import matplotlib.pyplot as plt

In [5]:
# Parameters
RSI_PERIOD = 14
STOCH_RSI_PERIOD = 14
RSI_OVERSOLD = 30
RSI_OVERBOUGHT = 70
STOCH_RSI_OVERBOUGHT = 0.8
STOCH_RSI_OVERSOLD = 0.2
initial_investment = 160_000_000

In [6]:
def calculate_indicators(df):
    if df.empty:
        return df
    
    # Calculate RSI
    rsi = ta.momentum.RSIIndicator(df['close'], RSI_PERIOD)
    df['RSI'] = rsi.rsi()
    
    # Calculate Stochastic RSI
    stoch_rsi = ta.momentum.StochRSIIndicator(df['close'], STOCH_RSI_PERIOD)
    df['Stoch_RSI'] = stoch_rsi.stochrsi()
    df['Stoch_RSI_Signal'] = stoch_rsi.stochrsi_d()
    
    df = df.dropna().reset_index(drop=True)
    
    return df

In [7]:
def generate_signals(df):
    if df.empty:
        return df
    
    # Initialize Signal column
    df['Signal'] = 0

    # Define constants
    RSI_OVERSOLD = 30
    RSI_OVERBOUGHT = 70
    STOCH_RSI_OVERSOLD = 0.2
    STOCH_RSI_OVERBOUGHT = 0.8

    # Buy signals: RSI exits oversold and Stoch RSI crosses above 0.2
    df.loc[(df['RSI'].shift(1) < RSI_OVERSOLD) & (df['RSI'] >= RSI_OVERSOLD) &
           (df['Stoch_RSI'].shift(1) < STOCH_RSI_OVERSOLD) & (df['Stoch_RSI'] >= STOCH_RSI_OVERSOLD), 'Signal'] = 1

    # Sell signals: RSI exits overbought and Stoch RSI crosses below 0.8
    df.loc[(df['RSI'].shift(1) > RSI_OVERBOUGHT) & (df['RSI'] <= RSI_OVERBOUGHT) &
           (df['Stoch_RSI'].shift(1) > STOCH_RSI_OVERBOUGHT) & (df['Stoch_RSI'] <= STOCH_RSI_OVERBOUGHT), 'Signal'] = -1

    return df

In [8]:
companies = [
    'SSI', 'BCM','VHM','VIC','VRE','BVH','POW','GAS','ACB','BID',
'CTG','HDB','MBB','SSB','SHB','STB','TCB','TPB','VCB','VIB','VPB','HPG',
'GVR','MSN','VNM','SAB','VJC','MWG','PLX','FPT']

In [9]:
def calculate_profit_total(df, initial_balance=160000000):
    if df.empty:
        return 0
    final_balance = df['Portfolio Value'].iloc[-1] 
    total_change = ((final_balance - initial_balance)/initial_balance) * 100
    return total_change

In [10]:
def backtest_strategy(df, initial_balance=160000000):
    if df.empty:
        return df
    
    balance = initial_balance
    shares = 0
    df['Portfolio Value'] = balance
    df['Shares'] = 0


    for i in range(1, len(df)):
        if df['Signal'].iloc[i] == 1:  # Buy signal
            if balance > 0: 
                shares_to_buy = balance // df['close'].iloc[i]
                balance -= shares_to_buy * df['close'].iloc[i]
                shares += shares_to_buy


        elif df['Signal'].iloc[i] == -1 and shares > 0:  # Sell signal
            balance += shares * df['close'].iloc[i]
            shares = 0

        df.loc[i, 'Portfolio Value'] = balance + shares * df['close'].iloc[i]
        df.loc[i, 'Shares'] = shares

    df.loc[len(df) - 1, 'Portfolio Value'] = balance + shares * df['close'].iloc[-1]

    return df

In [11]:
def test_company(symbol, start_date, end_date, resolution='1D', type='stock'):
    df = vn.stock_historical_data(symbol=symbol, 
                            start_date=start_date, 
                            end_date=end_date, resolution=resolution, type=type)
    if df.empty:
        return 0 
    df = calculate_indicators(df)
    df = generate_signals(df)
    df = backtest_strategy(df)
    total_profit_change = calculate_profit_total(df, initial_balance=160000000)
        
    return total_profit_change

In [12]:
START_DATE = '2021-01-01'
END_DATE = '2024-01-01'

In [13]:
profit_all = {}

for company in companies:
    try:
        company_data_all = test_company(company, start_date=START_DATE, end_date=END_DATE, resolution='1D', type='stock')
       
        # Update dictionaries
        profit_all[company] = company_data_all

    except Exception as e:
        print(f"Error occurred for {company}: {e}")

In [14]:
profit_all_df = pd.DataFrame.from_dict(profit_all, orient='index', columns=['Profit During all period']).reset_index()
final_result = profit_all_df.rename(columns={'index': 'Company'})
final_result.to_csv('result/rsi_storch_rsi.csv', index=False)